#<font color="yellow">Invoice Payment Collection Date Prediction</font>


This sample shows how to programmatically create Working Capital invoice documents in Kyriba and leverage Kyriba's Machine Learning (ML) capabilities to predict payment collection dates. The workflow consists of three main steps:

1. **Invoice Document Creation**: Fill out a form with invoice details and create the corresponding document in Kyriba.
2. **ML Prediction Processing**: Launch Kyriba's ML prediction process to analyze the created invoice and generate payment date prediction.
3. **Results Collection**: Retrieve the payment date forecast.

## <font color='yellow'> Step 0.1: Code for token generation and API management</font>

In [16]:
#@title Select **servicePack** the Sample should run
from subprocess import getstatusoutput
servicePack = "DEMO"

if servicePack == 'DEMO':
  branch = 'main'
else:
  branch = servicePack
print(branch)
rm = getstatusoutput("rm Colab_shared_code_for_API_usage -rf ")
clone = getstatusoutput("git clone -l -s --branch " +  branch + " https://github.com/kyriba/Colab_shared_code_for_API_usage.git")
import importlib
try:
  importlib.reload(sample_requests)
except:
  print()
from Colab_shared_code_for_API_usage import sample_requests

main


In [17]:
#@title Import custom modules
import json as js
from datetime import datetime, date
import pandas as pd
from time import sleep

## <font color='yellow'>Step 0.2: Connect and retrieve token</font>

In [26]:
#@title Connect and get token { display-mode: "form" }
token = sample_requests.login()

token 96cc028c-c4ce-4bda-bb41-43739369602a


## <font color='yellow'>Step 1: Create a Document </font>


In [27]:
#@title <font color='lime'>📝 Create a Document</font>

from dateutil import parser

# Get user input for invoice data
#@markdown Provide values for parameters below:
invoice_date = '2025-06-25' #@param {allow-input: true} {type:"date"}
due_date = '2025-07-25' #@param {allow-input: true} {type:"date"}
currency = 'EUR' #@param {allow-input: true} {type:"string"}
amount = 17554 #@param {allow-input: true} {type:"number"}
reference = 'INVOICE_1' #@param {allow-input: true} {type:"string"}

# Hardcoded values
buyer = "COMPANY01"
supplier = "THIRD-PARTY2"
supplier_reference = reference

# Create invoice data
invoice_data = {
    "Invoice Date": invoice_date,
    "Due Date": due_date,
    "Currency": currency,
    "Amount": amount,
    "Reference": reference,
    "Buyer": buyer,
    "Supplier": supplier,
    "Supplier Reference": supplier_reference
}

# ────────────────────────────────────────────────────────────────────────────
# Payload Builder Functions
# ────────────────────────────────────────────────────────────────────────────

def parse_eu_date(s: str):
    """Accepts 7/2/25, 07/02/2025, 7-2-25 … and returns a datetime."""
    return parser.parse(str(s), dayfirst=True, yearfirst=False)

def build_document_payload(data):
    amt = float(data["Amount"])

    return {
        "type": "INVOICE",
        "amount": amt,
        "currency": {
            "code": data["Currency"].strip().upper()
        },
        "buyer": {
            "uuid": "1e094c0d-20b1-45a2-e063-a80f11ac99de",
            "code": data["Buyer"].strip()
        },
        "thirdParty": {
            "code": data["Supplier"].strip()
        },
        "invoiceDate": parse_eu_date(data["Invoice Date"]).strftime("%Y-%m-%d"),
        "dueDate": parse_eu_date(data["Due Date"]).strftime("%Y-%m-%d"),
        "reference": str(data["Reference"]).strip(),
        "supplierReference": str(data["Supplier Reference"]).strip(),
        "documentCategory": "SELLER_MANAGED_RECEIVABLE",
        "paymentInfo": {
            "paymentPreparationInfo": {
                "paymentMethod": "TRANSFER",
                "debitAccount": {
                    "code": "000123"
                }
            }
        }
    }

# ────────────────────────────────────────────────────────────────────────────
# Create Document
# ────────────────────────────────────────────────────────────────────────────

assert 'token' in globals() and token, "⚠️ Run the Connect cell first to obtain a token."

# Create single document
payload = build_document_payload(invoice_data)
json_payload = sample_requests.js.dumps(payload)

doc_response = sample_requests.post_results(
    token,
    '/workcap/v2/documents',
    json_payload,
    {},
    {'Content-Type': 'application/json'}
)

doc_id = doc_response["uuid"]
print(f"✅ Created document {doc_id}")

# Store document info for later processing (as a single item instead of list)
created_document = {
    "document_id": doc_id,
    "reference": reference,
    "data": invoice_data
}


✅ Created document e0a54173-f0bc-4a4b-848a-24905439ec0f


## <font color='yellow'>Step 2: Run the Prediction </font>


In [28]:
#@title <font color='lime'>🔮 Run Prediction Template and Monitor Status</font>

sample_requests.run_process(token,'RF_AI_PREDICT', True)


 Run task bcff4708-5939-466c-b0f9-bef1b9dd5237
..........................................................
Complete

task details
{
  "uuid": "bcff4708-5939-466c-b0f9-bef1b9dd5237",
  "taskType": "Generate AI prediction data",
  "taskDescription": "Generate AI prediction data",
  "taskStatus": "Complete",
  "taskStartTime": "2025-07-10 15:57:23",
  "taskEndTime": "2025-07-10 15:57:54",
  "taskCode": "RF96_T",
  "taskLog": [
    "Thu 07/10/2025 3:57:23 PM <INFO> Start of task Generate AI prediction data based on Generate AI prediction data",
    "Thu 07/10/2025 3:57:23 PM <INFO> Document category; Entity; Documents;",
    "Thu 07/10/2025 3:57:54 PM <INFO> Seller managed receivable; THIRD-PARTY2; 55",
    "Thu 07/10/2025 3:57:54 PM <INFO> End of task Generate AI prediction data"
  ]
}


end task details


'{"status":"Complete","subTasks":[]}'

## <font color='yellow'>Step 3: Get the predicted payment date </font>

In [29]:
#@title <font color='lime'>📊 Fetch Prediction Results</font>

from time import sleep
from dateutil import parser
import json

# Get document info (single document)
doc_id = created_document["document_id"]
data = created_document["data"]

# Poll for predicted payment date
predicted = None

for attempt in range(30):
    try:
        # Use sample_requests to get document
        doc_response = sample_requests.get_results(
            token,
            f'/workcap/v2/documents?filter=uuid=={doc_id}'
        )

        if doc_response:
            try:
                parsed_doc = json.loads(doc_response)
                api_results = parsed_doc.get("results", [])

                if api_results and len(api_results) > 0:
                    prediction_data = api_results[0].get("prediction")

                    if prediction_data and isinstance(prediction_data, dict):
                        predicted = prediction_data.get("paymentDate")

                        if predicted:
                            break

            except json.JSONDecodeError:
                print(f"   Failed to parse JSON response")

        print(f"   No prediction yet, attempt {attempt + 1}/30")
        sleep(2)

    except Exception as e:
        print(f"   ⚠️  Error fetching prediction (attempt {attempt+1}): {e}")
        sleep(2)

status = "✅ Predicted" if predicted else "❌ Timeout"
print(f"\n{status} payment date = {predicted}")

# Create single result
result = {
    "DocumentId": doc_id,
    "Reference": data["Reference"],
    "Buyer": data["Buyer"],
    "Supplier": data["Supplier"],
    "Amount": data["Amount"],
    "Currency": data["Currency"],
    "Due Date": parse_eu_date(data["Due Date"]).strftime("%d/%m/%Y"),
    "Predicted Payment Date": parser.parse(predicted).strftime("%d/%m/%Y") if predicted else None,
}

# ────────────────────────────────────────────────────────────────────────────
# Display results
# ────────────────────────────────────────────────────────────────────────────

res_df = pd.DataFrame([result])

styled_df = res_df.style.set_properties(**{
    'text-align': 'center'
}).set_properties(
    subset=['Predicted Payment Date'],
    **{
        'background-color': 'lightgreen',
        'color': 'black',
        'text-align': 'center',
        'font-weight': 'bold'
    }
).set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]}
])

display(styled_df)

# Display summary
if predicted:
    due_date_obj = parse_eu_date(data["Due Date"])
    predicted_date_obj = parser.parse(predicted)
    difference = (predicted_date_obj - due_date_obj).days

else:
    print(f"\n❌ Summary: No prediction available for this document")


✅ Predicted payment date = 2025-07-22


,DocumentId,Reference,Buyer,Supplier,Amount,Currency,Due Date,Predicted Payment Date
0,e0a54173-f0bc-4a4b-848a-24905439ec0f,INVOICE_1,COMPANY01,THIRD-PARTY2,17554,EUR,25/07/2025,22/07/2025
